# Players Natioanlity Map

In [22]:
import pandas as pd
from geopy.geocoders import Nominatim
import folium
from collections import Counter


data = pd.read_csv('common_player_info.csv')


if 'country' not in data.columns:
    raise ValueError("CSV file must contain 'country' column.")


unique_countries = data['country'].unique()


geolocator = Nominatim(user_agent="geoapiExercises")


country_coords = {}

def get_lat_lon(country):
    try:
        location = geolocator.geocode(country)
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)
    except:
        return (None, None)


for country in unique_countries:
    country_coords[country] = get_lat_lon(country)


data['Coordinates'] = data['country'].apply(lambda x: country_coords[x])


data['Latitude'] = data['Coordinates'].apply(lambda coord: coord[0])
data['Longitude'] = data['Coordinates'].apply(lambda coord: coord[1])

data = data.dropna(subset=['Latitude', 'Longitude'])


location_counts = data['Coordinates'].value_counts().to_dict()
data['Count'] = data['Coordinates'].apply(lambda coord: location_counts[coord])


aggregated_data = data.groupby('Coordinates').agg({
    'country': 'first',
    'Count': 'first'
}).reset_index()


def get_color(density):
    if density > 100:
        return 'black'
    elif density > 50:
        return 'orange'
    elif density > 10:
        return '#C8102E'
    else:
        return '#1B3770'

aggregated_data['Color'] = aggregated_data['Count'].apply(get_color)


m = folium.Map(location=[0, 0], zoom_start=2) 

for _, row in aggregated_data.iterrows():
    if row['Coordinates'][0] is not None and row['Coordinates'][1] is not None:
        popup_text = f"{row['country']}: {row['Count']} players"
        popup = folium.Popup(popup_text, max_width=300, min_width=200) 
        folium.CircleMarker(
            location=[row['Coordinates'][0], row['Coordinates'][1]],
            radius=5,
            color=row['Color'],
            fill=True,
            fill_color=row['Color'],
            fill_opacity=0.6,
            popup=popup
        ).add_to(m)


#m.save('world_players_map_colored.html')

m


# US teams map

In [27]:
import pandas as pd
from geopy.geocoders import Nominatim
import folium
from collections import Counter


data = pd.read_csv('team.csv')


if 'city' not in data.columns or 'state' not in data.columns or 'full_name' not in data.columns:
    raise ValueError("CSV file must contain 'city', 'state', and 'team' columns.")


geolocator = Nominatim(user_agent="geoapiExercises")

def get_lat_lon(city, state):
    try:
        location = geolocator.geocode(f"{city}, {state}")
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)
    except:
        return (None, None)

data['Coordinates'] = data.apply(lambda row: get_lat_lon(row['city'], row['state']), axis=1)


data = data.dropna(subset=['Coordinates'])
data['Latitude'] = data['Coordinates'].apply(lambda coord: coord[0])
data['Longitude'] = data['Coordinates'].apply(lambda coord: coord[1])


location_counts = Counter(data['Coordinates'])
data['Count'] = data['Coordinates'].apply(lambda coord: location_counts[coord])


aggregated_data = data.groupby('Coordinates').agg({
    'city': 'first',
    'state': 'first',
    'Count': 'first',
    'full_name': lambda x: ', '.join(x)
}).reset_index()


def get_color(density):
    if density > 3:
        return 'black'
    elif density > 2:
        return 'orange'
    elif density > 1:
        return '#C8102E'
    else:
        return '#1B3770'

aggregated_data['Color'] = aggregated_data['Count'].apply(get_color)


m = folium.Map(location=[37.0902, -95.7129], zoom_start=5) 

for _, row in aggregated_data.iterrows():
    folium.CircleMarker(
        location=[row['Coordinates'][0], row['Coordinates'][1]],
        radius=5,
        color=row['Color'],
        fill=True,
        fill_color=row['Color'],
        fill_opacity=0.6,
        popup=f"{row['city']}, {row['state']}: {row['Count']} teams - {row['full_name']}"
    ).add_to(m)


#m.save('us_nba_teams_map_colored.html')


m


# NBA player Natioanlaity through the years
without the USA

In [8]:
import pandas as pd
import plotly.express as px
from ipywidgets import interact, IntSlider


df = pd.read_csv('common_player_info.csv')

def get_players_by_year_exclude_us(df, year):
    return df[(df['from_year'] <= year) & (df['to_year'] >= year) & (df['country'] != 'United States of America')]


def update_plot(year):
    filtered_df = get_players_by_year_exclude_us(df, year)
    country_count = filtered_df['country'].value_counts().reset_index()
    country_count.columns = ['country', 'count']
    
    fig = px.bar(country_count, x='country', y='count', title=f'NBA Players by Nationality in {year} (Excluding USA)')
    fig.show()


year_slider = IntSlider(min=df['from_year'].min(), max=df['to_year'].max(), step=1, description='Year')

interact(update_plot, year=year_slider)


interactive(children=(IntSlider(value=1946, description='Year', max=2023, min=1946), Output()), _dom_classes=(…

<function __main__.update_plot(year)>

# Avreage Height and Weight

In [20]:
import pandas as pd
import plotly.express as px


df = pd.read_csv('common_player_info.csv')


def convert_height(height):
    if isinstance(height, str):  
        feet, inches = map(int, height.split('-'))
        return (feet * 12 + inches)*2.54 
    else:
        return None  


df['height'] = df['height'].apply(convert_height)

df = df.dropna(subset=['height'])


df_avg = df.groupby('from_year')['height'].mean().reset_index()


fig = px.line(df_avg, x='from_year', y='height', labels={'from_year':'Year', 'height':'Average Height (in centimeters)'}, title='Average Height of NBA Players Over the Years')
fig.show()

In [21]:
df = pd.read_csv('common_player_info.csv')
def convert_height(weight):
    return weight*0.453592


df['weight'] = df['weight'].apply(convert_height)


df = df.dropna(subset=['weight'])


df_avg = df.groupby('from_year')['weight'].mean().reset_index()


fig = px.line(df_avg, x='from_year', y='weight', labels={'from_year':'Year', 'height':'Average weight (in KG)'}, title='Average weight of NBA Players Over the Years')
fig.show()